In [3]:
import ipywidgets as widgets
from IPython.display import display
from IPython.display import clear_output
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt


stock = pd.read_excel("10 Stocks & S&P500.xlsx")                 # read excel file
stock['Date'] = pd.to_datetime(stock['Date'], format='%Y-%m-%d') # Convert the date column to datetime format
clean_data = stock.columns[stock.notnull().all()]                # select data which is notnull
stock = stock[clean_data]                                        # stock return dataframe is cleaned
year_groups = stock.groupby(stock['Date'].dt.year)               # group by year

def Sharpe_Ratio(name):     # denote 'name' as the selected stock which we want to analyse & simplify the code to be one function
    
    #record selected stock and S&P columns
    columns_check =list(map(str.lower,list(stock.columns[1:])))  # create the list of columns name, and remove the date column
  
    for position in range(len(columns_check)):             # check the selected stock column  and S&P column
        if  name.lower() in columns_check[position]:       # if selected stock input is selected stock column
            selected_stock_column = position                   # record the column position index 
        if 'SP50'.lower() in columns_check[position]:      # if S&P 500  input  is in the S&P 500 full name
            SP50_column = position                             # record the S&P 500 column index
  
#Find Return 
    Ret = year_groups.mean()                               # ***average monthly return for each year which is not average annualised return****
    Ret = Ret[Ret.columns[1:]]*12                          # delete date column because dataframe with string cannot be multiplied by number, and annualise return by multiplying 12
    Ret1 = pd.DataFrame(Ret[Ret.columns[selected_stock_column]])  #select and create dataframe which has only selected stock return
    Ret2 = pd.DataFrame(Ret[Ret.columns[SP50_column]])     # select and create dataframe which has only S&P 500 return
    Ret_Final = pd.concat([Ret1, Ret2], axis='columns')    # concatenate two dataframes and keep the same row index
    Ret_Final = Ret_Final.rename_axis('Year').rename_axis('Return(%)', axis='columns') # change 'Date' axis name to 'Year', and name 'Return(%)' as column-axis name 
    
#Find Standard Deviation
    SD = year_groups.std()                                 # *** standard deviation each year***
    SD = SD[SD.columns[1:]]                                # remove date column
    SD = SD*np.sqrt(12)                                    # multiply sqrt(12) to annualise standard deviation
    SD1 = pd.DataFrame(SD[SD.columns[selected_stock_column]])  #select and create dataframe which has only selected stock standard deviation
    SD2 = pd.DataFrame(SD[SD.columns[SP50_column]])        # select and create dataframe which has only S&P 500 standard deviation
    SD_Final = pd.concat([SD1, SD2], axis='columns')       # concatenate two dataframes and keep the same row index
    SD_Final = SD_Final.rename_axis('Year').rename_axis('SD(%)', axis='columns') # change 'Date' axis name to 'Year', and name 'SD(%)' as column-axis name 
    
    
#Find Sharpe Ratio
    Rf = 3                                                 # assume risk-free rate = 3% per annum
    SR = (Ret_Final-Rf)/SD_Final                           # Sharpe Ratio = (Return - Risk-free rate)/standard deviation
    SR = SR.rename_axis('Sharpe Ratio', axis='columns')    # name 'Sharpe Ratio' as column-axis name
    
    display(SR)                                            # show Sharpe Ratio dataframe between selected stock & S&P 500
    display(SD_Final)                                      # show standard deviation dataframe between selected stock & S&P 500
    display(Ret_Final)                                     # show return dataframe between selected stock & S&P 500
    return SR

def SR_graph(stock_name, years):
    with output:
        clear_output(wait=True) # this clears out the previous results allowing for new data to prsented
        SR_data = Sharpe_Ratio(stock_name)# it enables us to call on the shapre ratio function that is calculated above by Don
        
        year_now = SR_data.index[-1]   #it must start at 2022 not 2023, pd.to_datetime('now').year refers the real current year (2023), but our latest data is 2022
        # get the latest index value from the create DataFrame
        
        year_back = year_now - years  # years variable come from slider, so analysed duration will be 2022 - years 
        #allows for the selection of the years that will be analysed 
        filtered_SR = SR_data[(SR_data.index >= year_back) & (SR_data.index <= year_now)] # only to include data from years back and years now
        
        plt.figure(figsize=(10, 5))
        if 'S&P 500 (SP50)' in filtered_SR.columns:
            plt.plot(filtered_SR.index, filtered_SR['S&P 500 (SP50)'], label='S&P500 SR') #plot S&P500 sharpe ratio
        else:
            print("'S&P 500' column not found in the DataFrame.")
        if stock_name != 'S&P 500 (SP50)':
            plt.plot(filtered_SR.index, filtered_SR[f"{stock_name}"], label=f"{stock_name} SR") # this filers and also plots the chosen stock for comparison with the s&p500
        
        plt.xlabel("Years")
        plt.ylabel('Sharpe Ratios')
        plt.title("Sharpe Ratio Comparison")
        plt.legend()
        plt.grid(True)
        plt.show()
       # all of the above plots the descriptions in the graph and allows us to presnet it with the show function 


#wideget for dropdown function
dropdown_control = widgets.Dropdown(
    options=[column for column in stock.columns if column != 'Date'],  # Exclude 'Date' column. define what will be included and displayed in the dropdown menu
    description='Select Stock:',
    ensure_option=True, # this makes sure that the dropdown always has something selected

    value=stock.columns[1] # selects the value from stock column and sets the default second column
)



#Create a Slider

slider_control = widgets.IntSlider(
    min=1, # min value of slider at 1
    max=10,# max value of slider at 10
    step=1, # makes it so the slider moves increments of 1
    description='years:',
    continous_update=False, # when the slider control is updated
    value=1 # set the slider to start at 1
)


# This function below controls what the slider and drop down function does in relation to stock selection and graph control
output = widgets.Output()


# widget control
def value_change(change):
     if change['type'] == 'change' and change['name'] == 'value':
            SR_graph(dropdown_control.value, slider_control.value)

dropdown_control.observe(value_change, names='value')
slider_control.observe(value_change, names='value')
# value_change function above helps to control th slider and dropdown menu, the value of the drop and slider will be changed, by calling the value_change function



SR_graph(dropdown_control.value, slider_control.value,)
# this ensure that the graph is displayed and show what the dropdown menu has selected. whilst alsso allowing for slider function to be tied with the graph


display(dropdown_control, slider_control, output)
# displays both the slider and dropdown function

Dropdown(description='Select Stock:', options=('Lockheed Martin Corporation (LMT-US)', 'Chevron Corporation (C…

IntSlider(value=1, description='years:', max=10, min=1)

Output()